### [ 손글씨 숫자 인식 모델 구현 ]
- 데이터셋 : mnist_train.csv, mnist_test.csv
- 학습종류 : 지동학습 - 다중클래스분류
- 학습방법 : 인공신경망기반 

[1] 모듈로딩 및 데이터 준비<hr>

In [38]:
# [1-1] 모듈로딩
import torch                                            # 텐서 및 수치과학 함수들 관련 모듈
import torch.nn as nn                                   # 신경망 층 관련 모듈
import torch.nn.functional as F                         # 신경망 함수들(AF, LF, MF) 모듈
import torch.optim as optim                             # 신경망 최적화 모듈


from torch.utils.data import Dataset, DataLoader        # pytorch의 데이터 로딩
from torch.utils.data import Subset                     # pytorch의 데이터셋 관련 모듈
from sklearn.model_selection import train_test_split
import pandas as pd


In [39]:
# [1-2] 데이터 준비
TRAIN_FILE = '../Data/mnist_train.csv'
TEST_FILE  = '../Data/mnist_test.csv'

## [1-3] 데이터 로딩
trainDF = pd.read_csv(TRAIN_FILE, header=None)
testDF  = pd.read_csv(TEST_FILE, header=None)


[2] 커스텀 데이터셋 준비 <hr>

In [40]:
## --------------------------------------------------------------------------------
## [2-1] 커스텀 데이터셋 클래스 정의
## --------------------------------------------------------------------------------
## 클래스이름 : ClfDataset
## 부모클래스 : Dataset
## 오버라이딩 : _ _init_ _(self)        : [필수] 피쳐, 타겟, [선택]행수, 컬럼수, 타겟 수...
##            _ _len_ _(self)          : len() 내장함수 실행 시 자동 호출, 샘플 수 반환
##            _ _getitem_ _(self, idx) : 인스턴스명[idx] 시 자동 호출,
##                                       idx에 해당하는 피쳐, 타겟을 텐서화 해서 반환
## --------------------------------------------------------------------------------
class ClfDataset(Dataset):

    ##- 피쳐와 타겟 저장 및 기타 속성 초기화 
    def __init__(self, dataDF):
        super().__init__()
        ## 피쳐, 타겟 초기화 필수
        self.x = dataDF[dataDF.columns[1:]].values
        self.y = dataDF[dataDF.columns[0]].values


    ##- 데이터 샘플 수 반환 메서드 : len() 함수에 자동호출됨
    def __len__(self):
        return self.x.shape[0] 
    
    ##- 인덱스에 해당하는 피쳐와 타겟 텐서 반환 메서드 : 인스턴스명[index]에 자동호출됨
    def __getitem__(self, index):
        xTS = torch.tensor(self.x[index], dtype=torch.float32)
        yTS = torch.tensor(self.y[index], dtype=torch.long)
        return xTS, yTS



In [41]:
## --------------------------------------------------------------------------------
## [2-2] 커스텀 데이터셋 인스턴스 생성 및 사용
## --------------------------------------------------------------------------------
allDS   = ClfDataset(trainDF)
testDS  = ClfDataset(testDF)

print(f'allDS : {len(allDS)},  testDS : {len(testDS)}')

allDS : 60000,  testDS : 10000


In [42]:
## [2-3] 학습용/검증용/테스트용 데이터셋 분리

## 학습용 데이터셋에서 타겟/라벨만 추출 
targetList = allDS.y 
dataIndexList =list( range(len(allDS)))

## 학습용/검증용 데이터셋 인덱스 분리
## -train_test_split() 함수 : train:test = 75:25 비율로 학습용, 테스트용 데이터셋 분리
##                            stratify : 분류용 데이터셋 경우 카테고리 비율 유지해서
##                                       데이터셋 분리
X_trainIdx, X_validIdx , y_train, y_valid = train_test_split( dataIndexList,
                                                              targetList,
                                                              train_size=0.8,
                                                              stratify=targetList,
                                                              random_state=10 )


In [43]:
## -----------------------------------------------------------------
## 학습용/검증용 데이터셋 생성 ===> Dataset ==> 2개 Subset 분리
## -----------------------------------------------------------------
trainDS = Subset(allDS, X_trainIdx)
validDS = Subset(allDS, X_validIdx)

print(f'allDS   : {type(allDS)},   {len(allDS)}개')
print(f'trainDS : {type(trainDS)}, {len(trainDS)}개')
print(f'validDS : {type(validDS)}, {len(validDS)}개')

allDS   : <class '__main__.ClfDataset'>,   60000개
trainDS : <class 'torch.utils.data.dataset.Subset'>, 48000개
validDS : <class 'torch.utils.data.dataset.Subset'>, 12000개


[3] 모델 클래스 만들기 <hr>

In [44]:
## --------------------------------------------------------------------
## [3-1] 커스텀 모델 클래스 정의
## --------------------------------------------------------------------
##
## --------------------------------------------------------------------
## 클래스이름 : MNISTModel
## 부모클래스 : nn.Module
## 오버라이딩 : __init__(self)
##            forward(self, x)
## 
## --------------------------------------------------------------------
class MNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.hd1_layer = nn.Linear(784, 128)
        self.hd2_layer = nn.Linear(128, 64)
        self.out_layer = nn.Linear(64, 10)
        
    def forward(self, x) :
        out = F.relu(self.hd1_layer(x))
        out = F.relu(self.hd2_layer(out))
        out = self.out_layer(out)         
        
        return out    
        

In [45]:
## --------------------------------------------------------------------
## [3-2] 커스텀 모델 클래스 구조 확인 -> torchinfo 활용
## --------------------------------------------------------------------
from torchinfo import summary

model = MNISTModel()
summary(model, input_size = (1, 784))

Layer (type:depth-idx)                   Output Shape              Param #
MNISTModel                               [1, 10]                   --
├─Linear: 1-1                            [1, 128]                  100,480
├─Linear: 1-2                            [1, 64]                   8,256
├─Linear: 1-3                            [1, 10]                   650
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
Total mult-adds (M): 0.11
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.44
Estimated Total Size (MB): 0.44

[4] 학습 준비 <hr>

In [46]:
## ------------------------------------------------------------
## [4-1] 학습 관련 설정
## ------------------------------------------------------------
## 학습 진행 횟수 및 학습량, 학습 진행 위치, W/b 업데이트 간격
EPOCHS      = 100
BATCH_SIZE  = 200
LR          = 0.01
DEVICE      = 'cuda' if torch.cuda.is_available() else "cpu"


In [47]:
## ------------------------------------------------------------
## [4-2] 학습 관련 인스턴스 생성
## ------------------------------------------------------------
## -> Model 인스턴스 : 자동으로 층별 W, b 텐서 생성 및 랜덤 초기화
model = MNISTModel().to(DEVICE)

## -> 손실함수 인스턴스 : 다중분류용
lossFn = nn.CrossEntropyLoss()

## -> 최적화 인스턴스 : 모델의 층별 파라미터 즉, W, b 업데이트
optim = optim.Adam(model.parameters(), lr= LR)

## -> 데이터로더 인스턴스 : 학습/검증/테스트에 사용될 학습량 만큼 데이터 추출
trainDL = DataLoader(trainDS, batch_size=BATCH_SIZE, shuffle=True)
validDL = DataLoader(trainDS, batch_size=BATCH_SIZE)
testDL = DataLoader(testDS, batch_size=BATCH_SIZE)


[5] 학습 진행 <hr>

In [48]:
import utils as uf

In [ ]:
### --> 학습 진행
## - 학습과 검증 결과 저장 : 학습 진행/중단 여부 결정, 모델 저장 여부
TRAIN_LA = {'loss' : [] , 'acc' : []}
VALID_LA = {'loss' : [] , 'acc' : []}

## 지정된 학습 횟수 만큼 학습 진행 & 에포크 단위로 학습과 검증 결과 저장
for epoch in range(EPOCHS):
    ## 1 에포크 학습
    train_loss, train_acc = uf.train_one_epoch(model, trainDL, lossFn, optim, DEVICE)
    
    ## 1에포크 학습 후 업데이트 W, b 검사 : 학습에 사용되지 않는 데이터
    valid_loss, valid_acc = uf.evaluate(model, validDL, lossFn, DEVICE)
    
    ## 학습과 검증 결과 저장
    TRAIN_LA['loss'].append(train_loss)
    TRAIN_LA['acc'].append(train_acc)
    VALID_LA['loss'].append(valid_loss)
    VALID_LA['acc'].append(valid_acc)
    
    ## 진행 상황 출력
    print(f'[EPOCH - {epoch:03}] Trian => Loss : {train_loss:.7f}   Acc : {train_acc:.5f}')
    print(f'{" "*14}VALID=> Loss : {train_loss:.7f}   Acc : {train_acc:.5f}')
    

[EPOCH - 000] Trian => Loss : 1.8336830   Acc : 0.80402
            VALID=> Loss : 1.8336830   Acc : 0.80402
[EPOCH - 001] Trian => Loss : 0.3208972   Acc : 0.91090
            VALID=> Loss : 0.3208972   Acc : 0.91090
[EPOCH - 002] Trian => Loss : 0.2805954   Acc : 0.92290
            VALID=> Loss : 0.2805954   Acc : 0.92290
[EPOCH - 003] Trian => Loss : 0.2622929   Acc : 0.92756
            VALID=> Loss : 0.2622929   Acc : 0.92756
[EPOCH - 004] Trian => Loss : 0.2495802   Acc : 0.93267
            VALID=> Loss : 0.2495802   Acc : 0.93267
[EPOCH - 005] Trian => Loss : 0.2297518   Acc : 0.93890
            VALID=> Loss : 0.2297518   Acc : 0.93890
[EPOCH - 006] Trian => Loss : 0.2474530   Acc : 0.93571
            VALID=> Loss : 0.2474530   Acc : 0.93571
[EPOCH - 007] Trian => Loss : 0.2237727   Acc : 0.94175
            VALID=> Loss : 0.2237727   Acc : 0.94175
[EPOCH - 008] Trian => Loss : 0.2227277   Acc : 0.94171
            VALID=> Loss : 0.2227277   Acc : 0.94171
[EPOCH - 009] Trian